In [ ]:
from random import randint
import math
import holoviews as hv
hv.extension('bokeh')
from bokeh.io import show, curdoc
from bokeh.layouts import layout
from bokeh.models import Slider, Button

### Funções Geométricas

In [ ]:
def create_points(n, min=0, max=50):
    points = []
    for _ in range(n):
        points.append((randint(min,max), randint(min,max)))
    return points

def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def slope(p1, p2):
    y = (p1[1] - p2[1])
    x = (p1[0] - p2[0])
    return math.atan2(y, x) 
    
def cross_product(p1, p2, p3):
    return ((p2[0] - p1[0]) * (p3[1] - p1[1])) - ((p2[1] - p1[1]) * (p3[0] - p1[0]))

### Funções de Animação

In [ ]:
renderer = hv.renderer('bokeh').instance(mode='server')
scatter = None
steps = None
end = None
dmap = None
stream = None

def ch(it):
    global steps
    global scatter
    return scatter * steps.get(it)

def modify_doc(doc):
    global dmap
    hvplot = renderer.get_plot(dmap, doc)

    def animate_update():
        it = slider.value + 1
        if it > end:
            it = start
        slider.value = it

    def slider_update(attrname, old, new):
        global stream
        stream.event(it=new)
    
    global end
    start = 0
    slider = Slider(start=start, end=end, value=start, step=1, title="Iteration")
    slider.on_change('value', slider_update)
    
    callback_id = None

    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = doc.add_periodic_callback(animate_update, 500)
        else:
            button.label = '► Play'
            doc.remove_periodic_callback(callback_id)
    button = Button(label='► Play', width=60)
    button.on_click(animate)
    
    plot = layout([
    [hvplot.state],
    [slider, button]], sizing_mode='fixed')
    
    doc.add_root(plot)
    return doc

In [ ]:
points = create_points(20)

### Varredura de Graham

In [ ]:
def graham_scan(points_):
    points = points_.copy()
    anchor = min(points, key=lambda p: (p[1], p[0]))
    points.pop(points.index(anchor))
    points.sort(key=lambda p: (slope(p, anchor), distance(anchor, p)))
    
    it = 0
    steps = {}
    hull = [anchor]
    steps[it] = hv.Path(hull) * hv.Path(hull)
    it += 1
    
    for p in points:
        steps[it] = hv.Path(hull) * hv.Path([hull[-1], p])
        it += 1
        hull.append(p)
        while len(hull) > 2 and cross_product(hull[-3], hull[-2], hull[-1]) < 0:
            hull.pop(-2)
            steps[it] = hv.Path(hull) * hv.Path([hull[-1], p])
            it += 1
            
    steps[it] = hv.Path(hull) * hv.Path([hull[-1], p])
    it += 1
    steps[it] = hv.Path(hull) * hv.Path([hull[-1], anchor])
    it += 1
    hull.append(hull[0])
    steps[it] = hv.Path(hull) * hv.Path(hull)
    
    return hull, steps

In [ ]:
def graham_plot(points):
    global scatter
    global steps
    global end
    global dmap
    global stream

    scatter = hv.Scatter(points)
    hull, steps = graham_scan(points)
    end = len(steps) - 1
    stream = hv.streams.Stream.define('Iteration', it=0)()
    dmap = hv.DynamicMap(ch, streams=[stream])

    show(modify_doc, notebook_url='localhost:8888')
    return scatter * hv.HoloMap(steps, kdims=['Iteration'])

graham_plot(points)

### Embrulho pra presente

In [ ]:
def gift_wrapping(points):
    anchor = min(points, key=lambda p: p[0])
    start = points.index(anchor)
    p = start
    hull = []
    it = 0
    steps = {}
    
    while 1:
        hull.append(points[p])
        steps[it] = hv.Path(hull) * hv.Path([hull[-1], points[p]])
        it += 1
        q = (p + 1) % len(points)
        for i in range(len(points)):
            steps[it] = hv.Path(hull) * hv.Path([hull[-1], points[i]])
            it += 1
            if cross_product(points[p], points[i], points[q]) < 0:
                q = i
        steps[it] = hv.Path(hull) * hv.Path([hull[-1], points[q]])
        it += 1
        p = q
        if p == start:
            break
    
    steps[it] = hv.Path(hull) * hv.Path([hull[-1], points[p]])
    it += 1
    steps[it] = hv.Path(hull) * hv.Path([hull[-1], anchor])
    it += 1
    hull.append(hull[0])
    steps[it] = hv.Path(hull) * hv.Path(hull)
    
    return hull, steps  

In [ ]:
def gift_plot(points):
    global scatter
    global steps
    global end
    global dmap
    global stream

    scatter = hv.Scatter(points)
    hull, steps = gift_wrapping(points)
    end = len(steps) - 1
    stream = hv.streams.Stream.define('Iteration', it=0)()
    dmap = hv.DynamicMap(ch, streams=[stream])

    show(modify_doc, notebook_url='localhost:8888')
    return scatter * hv.HoloMap(steps, kdims=['Iteration'])

gift_plot(points)

### Incremental

In [ ]:
# NOT WORKING

def incremental_v2(points_):
    points = points_.copy()
    points.sort(key=lambda p: p[0])
    hull = []
    for i in range(3):
        hull.append(points.pop(i))
    anchor = hull[-1]    
    hull.sort(key=lambda p: slope(p, anchor))
    
    for p in range(len(points)):
        n = len(hull)
        j = 0
        for i in range(n):
            if hull[i][0] > hull[j][0]:
                j = i
        
        u = j
        while cross_product(points[p], hull[u], hull[(u+1)%n]) >= 0:
            u = (u+1)%n
            
        l = j
        while cross_product(points[p], hull[l], hull[(n+l-1)%n]) <= 0:
            l = (n+l-1)%n
         
        new_hull = []
        visit = u
        while visit != l:
            visit = (visit+1)%n
            new_hull.append(hull[visit])
        
        new_hull.append(points[p])
        hull.clear()
        for i in range(len(new_hull)):
            hull.append(new_hull[i])
    
    return hull
    